In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Функция для создания радиусов станций в соответствии с годом постройки
def build_radius (entrance, radius):
    #Создание списка годов ввода станций
    list_year = entrance.Station_year_build.unique()
    list_year.sort()  
    
    Radius_year = pd.DataFrame(columns = ['Radius', 'Station_year_build', 'geometry'])
    
    #Построение радиуса
    y = 1
    for y in range(len(list_year)):
        #Перевод системы координат для построения буфера
        my_geoseries = entrance.loc[entrance.Station_year_build == list_year[y]].to_crs(epsg=2263)
        #Построение буфера
        coverage = my_geoseries.geometry.buffer(2*radius)
        #Возврат системы координат
        coverage = coverage.to_crs(epsg=4326)      
        #Построение общего буфера для радиуса
        my_union = coverage.geometry.unary_union
        #Определение геометрии входящей в общий буфер
        Radius_year.loc[y, 'Radius'] = radius
        Radius_year.loc[y,'Station_year_build'] = int(list_year[y])
        Radius_year.loc[y,'geometry'] = my_union
        
    Radius_year = gpd.GeoDataFrame(Radius_year, geometry = Radius_year.geometry)
    Radius_year.crs = 'epsg:4326'
    return Radius_year  

In [ ]:
#функция составления перечня домов с присвоением радиуса в котором они находятся
def build_spatial (build, entrance, radius):
    #формирование списка домов по радиусам
    build_list_by_radius = gpd.sjoin(build, build_radius (entrance, radius))
    build_list_by_radius_sort = build_list_by_radius.sort_values(by = ['Station_year_build'])
    #перезапись датафрейма с исключением повторяющихся домов в пересекающихся радиусах
    #остаются дома входящие в радиусы станций с наименьшим годом постройки
    build_list_final = gpd.GeoDataFrame(columns = build_list_by_radius_sort.columns)     
    j=0
    for j in range(len(build_list_by_radius_sort)):       
        if any(build_list_final.address == (build_list_by_radius_sort.iloc[j, :]).address) != True:
            build_list_final.loc[j, :] = build_list_by_radius_sort.iloc[j, :]  
    return build_list_final

In [ ]:
#Построение карты
def build_map (entrance, build):
    import folium
    from folium import Choropleth, Circle, Marker, GeoJson
    from folium.plugins import HeatMap, MarkerCluster
    import math    
    
    #создание основы с исходной точкой
    point = build.geometry.iloc[1]
    x, y = point.coords.xy
    m = folium.Map(location=[y[0], x[0]], zoom_start=11)
    #Перевод системы координат для построения буфера
    my_geoseries = entrance.to_crs(epsg=2263)
    #Построение буфера
    coverage = my_geoseries.geometry.buffer(2*(build.iloc[1, :]).Radius)
    #Возврат системы координат
    coverage = coverage.to_crs(epsg=4326)
    #Добавленине на карту
    GeoJson(coverage).add_to(m)
    
    #Добавление на карту домов
    mc = MarkerCluster()
    for idx, row in build.iterrows():
        if not math.isnan(row.geometry.x) and not math.isnan(row.geometry.y):
            mc.add_child(Marker([row.geometry.y, row.geometry.x]))
            m.add_child(mc)
    
    #build_total_map = build[["osm_id", "geometry"]].set_index("osm_id")
    #Choropleth(geo_data=build_total_map.__geo_interface__).add_to(m)
    
    return m

In [ ]:
#расчет площади текущих станций
def table_square (square):
    Square_total = pd.DataFrame(columns = ['Resedential'])
    i=0
    years=[]
    
    while i < (2022-(square.Station_year_build.min())):
        years.append((square.Station_year_build.min())+i)
        i=i+1
          
    for year in years:
        Res_square = square.loc[(square.Station_year_build <= year)
                                & (square.built_year <= year)]
        Square_total.loc[year] = [Res_square.area_residential.sum()]
        
    Square_total['year'] = Square_total.index
           
    return Square_total

In [ ]:
#расчет площади перпективных станций
def square_nst (square):
    Square_total = pd.DataFrame(columns = ['Resedential', 'Station_year_build'])
    
    i = 1   
    for i in range(int(square.Station_year_build.max()+1)):
        Res_square = square.loc[(square.Station_year_build == i)]
        Square_total.loc[i, 'Resedential'] = Res_square.area_residential.sum()
        Square_total.loc[i, 'Station_year_build'] = int(i)
           
    return Square_total

In [ ]:
#Поиск местоположения по адресу
from geopandas.tools import geocode

def my_geocoder(row):
    try:
        point = geocode(row, provider='arcgis').geometry.iloc[0]
        return pd.Series({'geometry': point})
    except:
        return None

In [ ]:
#Поиск вне диапазона
outside_range = build_960_new_spatial_Kazan.loc[~build_960_new_spatial_Kazan["geometry"].apply(lambda x: my_union.contains(x))]

In [ ]:
#Нижний Новгород. Данные Реформа ЖКХ
#Путь
build_RGKH_input_Novgorod = pd.read_csv('../input/d/dmitryushakov/kazan/build_Novgorod_RGKH.csv', sep=";")
#list(build_RGKH_input_Novgorod.columns)
#build_RGKH_input_Novgorod.head()

In [ ]:
#build_RGKH_input_Novgorod['address'][1]
#list((build_RGKH_input_Novgorod['formalname_city']).unique())
build_RGKH_Novgorod = build_RGKH_input_Novgorod.loc[build_RGKH_input_Novgorod.formalname_city == 'Нижний Новгород']
build_RGKH_Novgorod = build_RGKH_Novgorod.loc[:, ['formalname_city', 'built_year', 'area_residential', 'area_non_residential', 'area_total', 'address']]
build_RGKH_Novgorod = build_RGKH_Novgorod[pd.notnull(build_RGKH_Novgorod.built_year)]

build_RGKH_Novgorod['area_total'] = build_RGKH_Novgorod['area_total'].astype(str)
build_RGKH_Novgorod['area_total'] = build_RGKH_Novgorod['area_total'].apply(lambda x: x.replace(',', '.')).astype('float')
build_RGKH_Novgorod['area_non_residential'] = build_RGKH_Novgorod['area_non_residential'].astype(str)
build_RGKH_Novgorod['area_non_residential'] = build_RGKH_Novgorod['area_non_residential'].apply(lambda x: x.replace(',', '.')).astype('float')
build_RGKH_Novgorod['area_residential'] = build_RGKH_Novgorod['area_residential'].astype(str)
build_RGKH_Novgorod['area_residential'] = build_RGKH_Novgorod['area_residential'].apply(lambda x: x.replace(',', '.')).astype('float')

build_RGKH_Novgorod.head()

In [ ]:
build_RGKH_Novgorod.describe()

In [ ]:
#Получение кооридант зданий
build_RGKH_Novgorod[['geometry']] = build_RGKH_Novgorod.apply(lambda x: my_geocoder(x['address']), axis=1)

In [ ]:
build_RGKH_Novgorod.head()

In [ ]:
import geopandas as gpd
build_RGKH_Novgorod = gpd.GeoDataFrame(build_RGKH_Novgorod, geometry = build_RGKH_Novgorod.geometry)
build_RGKH_Novgorod = build_RGKH_Novgorod.set_crs('epsg:4326')
print(build_RGKH_Novgorod.crs)

In [ ]:
#Нижний Новгород. Данные для пространственной оценки
import geopandas as gpd
#Путь
entrance_filepath_Novgorod = "../input/estimate-metro-osm/Entrances_Novgorod.gpkg"
# Запись в таблицы
entrance_Novgorod = gpd.read_file(entrance_filepath_Novgorod)

#entrance_Novgorod.head()

In [ ]:
build_400_spatial_Novgorod = build_spatial (build_RGKH_Novgorod, entrance_Novgorod, 400)
build_400_spatial_Novgorod.head()

In [ ]:
build_map_Novgorod = build_map (entrance_Novgorod, build_400_spatial_Novgorod)
build_map_Novgorod

In [ ]:
build_400_square_Novgorod = table_square (build_400_spatial_Novgorod)
build_400_square_Novgorod.to_csv('build_400_square_Novgorod.csv', index=False)

In [ ]:
build_800_spatial_Novgorod = build_spatial (build_RGKH_Novgorod, entrance_Novgorod, 800)
build_800_square_Novgorod = table_square (build_800_spatial_Novgorod)
build_800_square_Novgorod.to_csv('build_800_square_Novgorod.csv', index=False)

In [ ]:
build_1200_spatial_Novgorod = build_spatial (build_RGKH_Novgorod, entrance_Novgorod, 1200)
build_1200_square_Novgorod = table_square (build_1200_spatial_Novgorod)
build_1200_square_Novgorod.to_csv('build_1200_square_Novgorod.csv', index=False)

In [ ]:
build_square_Novgorod = gpd.GeoDataFrame(columns = ['year', '400', '400_800', '800_1200'])
build_square_Novgorod['year'] = build_400_square_Novgorod['year']
build_square_Novgorod['400'] = build_400_square_Novgorod['Resedential']
build_square_Novgorod['400_800'] = build_800_square_Novgorod['Resedential'] - build_400_square_Novgorod['Resedential']
build_square_Novgorod['800_1200'] = build_1200_square_Novgorod['Resedential'] - build_800_square_Novgorod['Resedential']
build_square_Novgorod.to_csv('build_square_Novgorod.csv', index=False)

In [ ]:
#Самара. Данные Реформа ЖКХ
build_Samara_RGKH_input = pd.read_csv('../input/d/dmitryushakov/kazan/build_Samara_RGKH.csv', sep=";")
#Подготовка данных
build_Samara_RGKH = build_Samara_RGKH_input.loc[build_Samara_RGKH_input.formalname_city == 'Самара']
build_Samara_RGKH = build_Samara_RGKH.loc[:, ['formalname_city', 'built_year', 'area_residential', 'area_non_residential', 'area_total', 'address']]
build_Samara_RGKH = build_Samara_RGKH[pd.notnull(build_Samara_RGKH.built_year)]

build_Samara_RGKH['area_total'] = build_Samara_RGKH['area_total'].astype(str)
build_Samara_RGKH['area_total'] = build_Samara_RGKH['area_total'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Samara_RGKH['area_non_residential'] = build_Samara_RGKH['area_non_residential'].astype(str)
build_Samara_RGKH['area_non_residential'] = build_Samara_RGKH['area_non_residential'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Samara_RGKH['area_residential'] = build_Samara_RGKH['area_residential'].astype(str)
build_Samara_RGKH['area_residential'] = build_Samara_RGKH['area_residential'].apply(lambda x: x.replace(',', '.')).astype('float')

build_Samara_RGKH.head()

In [ ]:
build_Samara_RGKH.describe()

In [ ]:
build_Samara_RGKH[['geometry']] = build_Samara_RGKH.apply(lambda x: my_geocoder(x['address']), axis=1)

In [ ]:
build_Samara_RGKH.head()

In [ ]:
import geopandas as gpd
build_Samara_RGKH = gpd.GeoDataFrame(build_Samara_RGKH, geometry = build_Samara_RGKH.geometry)
build_Samara_RGKH = build_Samara_RGKH.set_crs('epsg:4326')
print(build_Samara_RGKH.crs)

In [ ]:
#Нижний Новгород. Данные для пространственной оценки
import geopandas as gpd
#Путь
entrance_filepath_Samara = "../input/estimate-metro-osm/Entrances_Samara.gpkg"
# Запись в таблицы
entrance_Samara = gpd.read_file(entrance_filepath_Samara)
entrance_Samara.head()

In [ ]:
build_400_spatial_Samara = build_spatial (build_Samara_RGKH, entrance_Samara, 400)
build_400_square_Samara = table_square (build_400_spatial_Samara)
build_400_square_Samara.to_csv('build_400_square_Samara.csv', index=False)
#build_400_square_Samara.head()

In [ ]:
build_800_spatial_Samara = build_spatial (build_Samara_RGKH, entrance_Samara, 800)
build_800_square_Samara = table_square (build_800_spatial_Samara)
#build_800_square_Samara.to_csv('build_800_square_Samara.csv', index=False)
build_800_square_Samara.head()

In [ ]:
build_square_Samara = gpd.GeoDataFrame(columns = ['year', '400', '400_800', '800_1200'])
build_square_Samara['year'] = build_400_square_Samara['year']
build_square_Samara['400'] = build_400_square_Samara['Resedential']
build_square_Samara['400_800'] = build_800_square_Samara['Resedential'] - build_400_square_Samara['Resedential']
build_square_Samara['800_1200'] = build_1200_square_Samara['Resedential'] - build_800_square_Samara['Resedential']
build_square_Samara.to_csv('build_square_Samara.csv', index=False)

In [ ]:
#Новосибирск. Данные Реформа ЖКХ
build_Novosibirsk_RGKH_input = pd.read_csv('../input/d/dmitryushakov/kazan/build_Novosibirsk_RGKH.csv', sep=";")
#Подготовка данных
build_Novosibirsk_RGKH = build_Novosibirsk_RGKH_input.loc[build_Novosibirsk_RGKH_input.formalname_city == 'Новосибирск']
build_Novosibirsk_RGKH = build_Novosibirsk_RGKH.loc[:, ['formalname_city', 'built_year', 'area_residential', 'area_non_residential', 'area_total', 'address']]
build_Novosibirsk_RGKH = build_Novosibirsk_RGKH[pd.notnull(build_Novosibirsk_RGKH.built_year)]

build_Novosibirsk_RGKH['area_total'] = build_Novosibirsk_RGKH['area_total'].astype(str)
build_Novosibirsk_RGKH['area_total'] = build_Novosibirsk_RGKH['area_total'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Novosibirsk_RGKH['area_non_residential'] = build_Novosibirsk_RGKH['area_non_residential'].astype(str)
build_Novosibirsk_RGKH['area_non_residential'] = build_Novosibirsk_RGKH['area_non_residential'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Novosibirsk_RGKH['area_residential'] = build_Novosibirsk_RGKH['area_residential'].astype(str)
build_Novosibirsk_RGKH['area_residential'] = build_Novosibirsk_RGKH['area_residential'].apply(lambda x: x.replace(',', '.')).astype('float')

build_Novosibirsk_RGKH.head()

In [ ]:
build_Novosibirsk_RGKH.describe()

In [ ]:
build_Novosibirsk_RGKH[['geometry']] = build_Novosibirsk_RGKH.apply(lambda x: my_geocoder(x['address']), axis=1)

In [ ]:
build_Novosibirsk_RGKH.head()

In [ ]:
import geopandas as gpd
build_Novosibirsk_RGKH = gpd.GeoDataFrame(build_Novosibirsk_RGKH, geometry = build_Novosibirsk_RGKH.geometry)
build_Novosibirsk_RGKH = build_Novosibirsk_RGKH.set_crs('epsg:4326')
print(build_Novosibirsk_RGKH.crs)

In [ ]:
#Нижний Новгород. Данные для пространственной оценки
import geopandas as gpd
#Путь
entrance_filepath_Novosibirsk = "../input/estimate-metro-osm/Entrances_Novosibirsk.gpkg"
# Запись в таблицы
entrance_Novosibirsk = gpd.read_file(entrance_filepath_Novosibirsk)
entrance_Novosibirsk.head()

In [ ]:
build_400_spatial_Novosibirsk = build_spatial (build_Novosibirsk_RGKH, entrance_Novosibirsk, 400)
build_400_square_Novosibirsk = table_square (build_400_spatial_Novosibirsk)
build_400_square_Novosibirsk.to_csv('build_400_square_Novosibirsk.csv', index=False)
#build_400_square_Novosibirsk.head()

In [ ]:
build_800_spatial_Novosibirsk = build_spatial (build_Novosibirsk_RGKH, entrance_Novosibirsk, 800)
build_800_square_Novosibirsk = table_square (build_800_spatial_Novosibirsk)
build_800_square_Novosibirsk.to_csv('build_800_square_Novosibirsk.csv', index=False)
#build_800_square_Novosibirsk.head()

In [ ]:
build_1200_spatial_Novosibirsk = build_spatial (build_Novosibirsk_RGKH, entrance_Novosibirsk, 1200)
build_1200_square_Novosibirsk = table_square (build_1200_spatial_Novosibirsk)
build_1200_square_Novosibirsk.to_csv('build_1200_square_Novosibirsk.csv', index=False)
#build_1200_square_Novosibirsk.head()

In [ ]:
build_square_Novosibirsk = gpd.GeoDataFrame(columns = ['year', '400', '400_800', '800_1200'])
build_square_Novosibirsk['year'] = build_400_square_Novosibirsk['year']
build_square_Novosibirsk['400'] = build_400_square_Novosibirsk['Resedential']
build_square_Novosibirsk['400_800'] = build_800_square_Novosibirsk['Resedential'] - build_400_square_Novosibirsk['Resedential']
build_square_Novosibirsk['800_1200'] = build_1200_square_Novosibirsk['Resedential'] - build_800_square_Novosibirsk['Resedential']
build_square_Novosibirsk.to_csv('build_square_Novosibirsk.csv', index=False)

In [ ]:
#Екатеринбург. Данные Реформа ЖКХ
build_Ekaterinburg_RGKH_input = pd.read_csv('../input/d/dmitryushakov/kazan/build_Ekaterinburg_RGKH.csv', sep=";")
#Подготовка данных
build_Ekaterinburg_RGKH = build_Ekaterinburg_RGKH_input.loc[build_Ekaterinburg_RGKH_input.formalname_city == 'Екатеринбург']
build_Ekaterinburg_RGKH = build_Ekaterinburg_RGKH.loc[:, ['formalname_city', 'built_year',
                                                          'area_residential', 'area_non_residential', 'area_total', 'address']]
build_Ekaterinburg_RGKH = build_Ekaterinburg_RGKH[pd.notnull(build_Ekaterinburg_RGKH.built_year)]

build_Ekaterinburg_RGKH['area_total'] = build_Ekaterinburg_RGKH['area_total'].astype(str)
build_Ekaterinburg_RGKH['area_total'] = build_Ekaterinburg_RGKH['area_total'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Ekaterinburg_RGKH['area_non_residential'] = build_Ekaterinburg_RGKH['area_non_residential'].astype(str)
build_Ekaterinburg_RGKH['area_non_residential'] = build_Ekaterinburg_RGKH['area_non_residential'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Ekaterinburg_RGKH['area_residential'] = build_Ekaterinburg_RGKH['area_residential'].astype(str)
build_Ekaterinburg_RGKH['area_residential'] = build_Ekaterinburg_RGKH['area_residential'].apply(lambda x: x.replace(',', '.')).astype('float')

build_Ekaterinburg_RGKH.head()

In [ ]:
build_Ekaterinburg_RGKH.describe()

In [ ]:
build_Ekaterinburg_RGKH[['geometry']] = build_Ekaterinburg_RGKH.apply(lambda x: my_geocoder(x['address']), axis=1)

In [ ]:
build_Ekaterinburg_RGKH.head()

In [ ]:
import geopandas as gpd
build_Ekaterinburg_RGKH = gpd.GeoDataFrame(build_Ekaterinburg_RGKH, geometry = build_Ekaterinburg_RGKH.geometry)
build_Ekaterinburg_RGKH = build_Ekaterinburg_RGKH.set_crs('epsg:4326')
print(build_Ekaterinburg_RGKH.crs)

In [ ]:
#Нижний Новгород. Данные для пространственной оценки
import geopandas as gpd
#Путь
entrance_filepath_Ekaterinburg = "../input/estimate-metro-osm/Entrances_Ekaterinburg.gpkg"
# Запись в таблицы
entrance_Ekaterinburg = gpd.read_file(entrance_filepath_Ekaterinburg)
entrance_Ekaterinburg.head()

In [ ]:
build_400_spatial_Ekaterinburg = build_spatial (build_Ekaterinburg_RGKH, entrance_Ekaterinburg, 400)
build_400_square_Ekaterinburg = table_square (build_400_spatial_Ekaterinburg)
build_400_square_Ekaterinburg.to_csv('build_400_square_Ekaterinburg.csv', index=False)
#build_400_square_Ekaterinburg.head()

In [ ]:
build_map_Ekaterinburg = build_map (entrance_Ekaterinburg, build_400_spatial_Ekaterinburg)
build_map_Ekaterinburg

In [ ]:
build_800_spatial_Ekaterinburg = build_spatial (build_Ekaterinburg_RGKH, entrance_Ekaterinburg, 800)
build_800_square_Ekaterinburg = table_square (build_800_spatial_Ekaterinburg)
build_800_square_Ekaterinburg.to_csv('build_800_square_Ekaterinburg.csv', index=False)
#build_800_square_Ekaterinburg.head()

In [ ]:
build_map_Ekaterinburg = build_map (entrance_Ekaterinburg, build_800_spatial_Ekaterinburg)
build_map_Ekaterinburg

In [ ]:
build_1200_spatial_Ekaterinburg = build_spatial (build_Ekaterinburg_RGKH, entrance_Ekaterinburg, 1200)
build_1200_square_Ekaterinburg = table_square (build_1200_spatial_Ekaterinburg)
build_1200_square_Ekaterinburg.to_csv('build_1200_square_Ekaterinburg.csv', index=False)
#build_1200_square_Ekaterinburg.head()

In [ ]:
build_square_Ekaterinburg = gpd.GeoDataFrame(columns = ['year', '400', '400_800', '800_1200'])
build_square_Ekaterinburg['year'] = build_400_square_Ekaterinburg['year']
build_square_Ekaterinburg['400'] = build_400_square_Ekaterinburg['Resedential']
build_square_Ekaterinburg['400_800'] = build_800_square_Ekaterinburg['Resedential'] - build_400_square_Ekaterinburg['Resedential']
build_square_Ekaterinburg['800_1200'] = build_1200_square_Ekaterinburg['Resedential'] - build_800_square_Ekaterinburg['Resedential']
build_square_Ekaterinburg.to_csv('build_square_Ekaterinburg.csv', index=False)

In [ ]:
#Казань. Данные Реформа ЖКХ
build_Kazan_RGKH_input = pd.read_csv('../input/d/dmitryushakov/kazan/build_Kazan_RGKH.csv', sep=";")
#Подготовка данных
build_Kazan_RGKH = build_Kazan_RGKH_input.loc[build_Kazan_RGKH_input.formalname_city == 'Казань']
build_Kazan_RGKH = build_Kazan_RGKH.loc[:, ['formalname_city', 'built_year',
                                                          'area_residential', 'area_non_residential', 'area_total', 'address']]
build_Kazan_RGKH = build_Kazan_RGKH[pd.notnull(build_Kazan_RGKH.built_year)]

build_Kazan_RGKH['area_total'] = build_Kazan_RGKH['area_total'].astype(str)
build_Kazan_RGKH['area_total'] = build_Kazan_RGKH['area_total'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Kazan_RGKH['area_non_residential'] = build_Kazan_RGKH['area_non_residential'].astype(str)
build_Kazan_RGKH['area_non_residential'] = build_Kazan_RGKH['area_non_residential'].apply(lambda x: x.replace(',', '.')).astype('float')
build_Kazan_RGKH['area_residential'] = build_Kazan_RGKH['area_residential'].astype(str)
build_Kazan_RGKH['area_residential'] = build_Kazan_RGKH['area_residential'].apply(lambda x: x.replace(',', '.')).astype('float')

build_Kazan_RGKH.head()

In [ ]:
build_Kazan_RGKH.describe()

In [ ]:
build_Kazan_RGKH[['geometry']] = build_Kazan_RGKH.apply(lambda x: my_geocoder(x['address']), axis=1)

In [ ]:
build_Kazan_RGKH.head()

In [ ]:
import geopandas as gpd
build_Kazan_RGKH = gpd.GeoDataFrame(build_Kazan_RGKH, geometry = build_Kazan_RGKH.geometry)
build_Kazan_RGKH = build_Kazan_RGKH.set_crs('epsg:4326')
print(build_Kazan_RGKH.crs)

In [ ]:
#Нижний Новгород. Данные для пространственной оценки
import geopandas as gpd
#Путь
entrance_filepath_Kazan = "../input/estimate-metro-osm/Entrance_Kazan.gpkg"
# Запись в таблицы
entrance_Kazan = gpd.read_file(entrance_filepath_Kazan)
entrance_Kazan.head()

In [ ]:
build_400_spatial_Kazan = build_spatial (build_Kazan_RGKH, entrance_Kazan, 400)
build_400_square_Kazan = table_square (build_400_spatial_Kazan)
build_400_square_Kazan.to_csv('build_400_square_Kazan.csv', index=False)
#build_400_square_Kazan.head()

In [ ]:
build_800_spatial_Kazan = build_spatial (build_Kazan_RGKH, entrance_Kazan, 800)
build_800_square_Kazan = table_square (build_800_spatial_Kazan)
build_800_square_Kazan.to_csv('build_800_square_Kazan.csv', index=False)
#build_800_square_Kazan.head()

In [ ]:
build_1200_spatial_Kazan = build_spatial (build_Kazan_RGKH, entrance_Kazan, 1200)
build_1200_square_Kazan = table_square (build_1200_spatial_Kazan)
build_1200_square_Kazan.to_csv('build_1200_square_Kazan.csv', index=False)
#build_1200_square_Kazan.head()

In [ ]:
build_map_Kazan = build_map (entrance_Kazan, build_1200_spatial_Kazan)
build_map_Kazan

In [ ]:
build_square_Kazan = gpd.GeoDataFrame(columns = ['year', '400', '400_800', '800_1200'])
build_square_Kazan['year'] = build_400_square_Kazan['year']
build_square_Kazan['400'] = build_400_square_Kazan['Resedential']
build_square_Kazan['400_800'] = build_800_square_Kazan['Resedential'] - build_400_square_Kazan['Resedential']
build_square_Kazan['800_1200'] = build_1200_square_Kazan['Resedential'] - build_800_square_Kazan['Resedential']
build_square_Kazan.to_csv('build_square_Kazan.csv', index=False)